Similaridade de strings
* Data cleaning
* Correção de digitação
* Tradução de idiomas

In [1]:
!pip install fuzzywuzzy

In [ ]:
!pip install fuzzywuzzy[speedup]

In [1]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

E:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [6]:
s1 = 'Doença Cardiovascular'
s2 = 'Doença Cardiovascular'

In [7]:
fuzz.ratio(s1,s2)

100

In [16]:
s1 = 'Doença Cardiovascular'
s2 = 'Doença Cardiovasculhar'

In [17]:
fuzz.ratio(s1,s2)

98

In [13]:
s1 = 'Doença Cardiovascular'
s2 = 'Doença Cardiovasculhar!!'
fuzz.ratio(s1,s2)

93

Similaridade parcial

In [18]:
s1 = 'Doença Cardiovascular'
s2 = '###$$%S!Doença Cardiovascular#$#$#^^^^^!!'
fuzz.ratio(s1,s2)

68

In [19]:
s1 = 'Doença Cardiovascular'
s2 = '###$$%S!Doença Cardiovascular#$#$#^^^^^!!'
fuzz.partial_ratio(s1,s2)

100

In [20]:
s1 = 'Doença Cardiovascular'
s2 = '###$$%S!Doença Cardiovasculhar#$#$#^^^^^!!'
fuzz.partial_ratio(s1,s2)

95

Ordem de caracteres diferentes
partial_token_sort_ration - separa os tokens por espaço e ordena por ordem alfabética
coloca as strings em minúsculas
considera apenas as strings consultadas

In [22]:
s1 = 'Doença Cardiovascular'
s2 = 'Cardiovascular Doença'
fuzz.ratio(s1,s2)

67

In [24]:
s1 = 'Doença Cardiovascular'
s2 = 'Cardiovascular Doença'
fuzz.partial_token_sort_ratio(s1,s2)

100

In [25]:
s1 = 'Doença Cardiovascular'
s2 = 'cardiovascular doença'
fuzz.partial_token_sort_ratio(s1,s2)

100

In [26]:
s1 = 'Doença Cardiovascular'
s2 = '%%%%%cardiovascular doença&&&&****@@@'
fuzz.partial_token_sort_ratio(s1,s2)

100

Processando uma lista de strings
* Aplicar o fuzzywuzzy para corrigir strings em uma base de dados

In [30]:
from fuzzywuzzy import process

In [28]:
lista = ['Doença Cardiovascular', 'doença cardiovascular!!', 'Doenca Cardiovascular', 'Doenc. Cardio']

In [29]:
lista

['Doença Cardiovascular',
 'doença cardiovascular!!',
 'Doenca Cardiovascular',
 'Doenc. Cardio']

In [31]:
process.extract('Doença cardiovascular', lista, scorer=fuzz.partial_ratio)

[('Doença Cardiovascular', 100),
 ('doença cardiovascular!!', 100),
 ('Doenca Cardiovascular', 95),
 ('Doenc. Cardio', 85)]

In [32]:
process.extract('Doença cardiovascular', lista, scorer=fuzz.partial_ratio,limit=3)

[('Doença Cardiovascular', 100),
 ('doença cardiovascular!!', 100),
 ('Doenca Cardiovascular', 95)]

In [34]:
process.extractOne('Doença cardiovascular', lista, scorer=fuzz.partial_ratio, score_cutoff=95)

('Doença Cardiovascular', 100)

Data cleaning em um DataFrame

In [35]:
import pandas as pd
from collections import OrderedDict
data = OrderedDict(
    {
        'codigo_produto': [10,11,12,13,14],
        'descricao': ['iphone 6ss', 'iphone 6s', 'iphoni 6s', 'ipone 6s', 'Iphone 6s,,,,']
    }
)

In [36]:
data

OrderedDict([('codigo_produto', [10, 11, 12, 13, 14]),
             ('descricao',
              ['iphone 6ss',
               'iphone 6s',
               'iphoni 6s',
               'ipone 6s',
               'Iphone 6s,,,,'])])

In [37]:
dataset = pd.DataFrame(data)

In [38]:
dataset.head()

,codigo_produto,descricao
0,10,iphone 6ss
1,11,iphone 6s
2,12,iphoni 6s
3,13,ipone 6s
4,14,"Iphone 6s,,,,"


In [45]:
process.extractOne('Iphone 6s', choices=dataset.descricao, scorer=fuzz.ratio, score_cutoff=95)

('iphone 6s', 100, 1)

In [46]:
def AplicaFuzzy(query, dados, metodo_ratio, score_corte):
    return process.extractOne(query,choices=dados,scorer=metodo_ratio,score_cutoff=score_corte)

In [47]:
AplicaFuzzy('Iphone 6s', dataset.descricao, fuzz.ratio, 95)

('iphone 6s', 100, 1)

In [48]:
dataset['descricao2'] = AplicaFuzzy('Iphone 6s', dataset.descricao, fuzz.ratio, 95)[0]

In [49]:
dataset

,codigo_produto,descricao,descricao2
0,10,iphone 6ss,iphone 6s
1,11,iphone 6s,iphone 6s
2,12,iphoni 6s,iphone 6s
3,13,ipone 6s,iphone 6s
4,14,"Iphone 6s,,,,",iphone 6s
